Load pipeline

In [3]:
from optimum.onnxruntime import ORTStableDiffusionPipeline
model_id = "yahoo-inc/photo-background-generation"
pipeline = ORTStableDiffusionPipeline.from_pretrained(model_id,  export=True)
pipeline.save_pretrained("./onnx-photo-background-generation")

Framework not specified. Using pt to export the model.
Keyword arguments {'subfolder': '', 'use_auth_token': None, 'trust_remote_code': False} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

C:\Users\sasha\anaconda3\envs\DS-study\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

***** Exporting submodel 1/4: CLIPTextModel *****
Using framework PyTorch: 2.2.2
C:\Users\sasha\anaconda3\envs\DS-study\Lib\site-packages\transformers\modeling_attn_mask_utils.py:86: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
C:\Users\sasha\anaconda3\envs\DS-study\Lib\site-packages\transformers\modeling_attn_mask_utils.py:162: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be inc

In [1]:


from diffusers import DiffusionPipeline
model_id = "yahoo-inc/photo-background-generation"
pipeline = DiffusionPipeline.from_pretrained(model_id, custom_pipeline=model_id)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
pipeline = pipeline.to('cuda')

ValueError: Asked to use CUDAExecutionProvider as an ONNX Runtime execution provider, but the available execution providers are ['AzureExecutionProvider', 'CPUExecutionProvider'].

In [5]:
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# device = torch.device("cpu")
print(f"Device: {device}, {use_cuda}")

Device: cuda, True


Load an image and extract its background and foreground

In [4]:
from PIL import Image, ImageOps
import requests
from io import BytesIO
from transparent_background import Remover

def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    # print(img.size)
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)

seed = 0
# image_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/16/Granja_comary_Cisne_-_Escalavrado_e_Dedo_De_Deus_ao_fundo_-Teres%C3%B3polis.jpg/2560px-Granja_comary_Cisne_-_Escalavrado_e_Dedo_De_Deus_ao_fundo_-Teres%C3%B3polis.jpg'
# image_url = 'https://drive.google.com/drive/u/0/folders/1VDCasP94xBVLd_WE1AVNgsjd18tBzxBG/Granja_comary_Cisne_-_Escalavrado_e_Dedo_De_Deus_ao_fundo_-Teres%C3%B3polis.jpg'
# response = requests.get(image_url)
# print(response)
# print(response.content)
# img = Image.open(BytesIO(content))

# read an image from bat.jpg
img = Image.open('bat.jpg')
img = resize_with_padding(img, (512, 512))
print(img)

# Load background detection model
remover = Remover() # default setting
remover = Remover(mode='base') # nightly release checkpoint

# Get foreground mask
fg_mask = remover.process(img, type='map') # default setting - transparent background

<PIL.Image.Image image mode=RGB size=512x512 at 0x21181E98090>


C:\Users\sasha\anaconda3\envs\DS-study\Lib\site-packages\torch\functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Settings -> Mode=base, Device=cuda:0, Torchscript=disabled
Settings -> Mode=base, Device=cuda:0, Torchscript=disabled


Background generation

In [14]:
seed = 5
mask = ImageOps.invert(fg_mask)
img = resize_with_padding(img, (512, 512))
generator = (torch.Generator(device='cuda')
             # .manual_seed(seed)
             )
# prompt = 'A dark swan on a beach'
# prompt = 'A radiator in the living room with grey-blue walls with maximalist style.'
prompt = 'A radiator near the wall of the kitchen with a chair nearby.'
# prompt = 'Батарея в гостиной с голубыми стенами'
cond_scale = 1.0
with torch.autocast("cuda"):
    controlnet_image = pipeline(
        prompt=prompt, image=img, mask_image=mask, control_image=mask, num_images_per_prompt=1, generator=generator, num_inference_steps=20, guess_mode=False, controlnet_conditioning_scale=cond_scale
    ).images[0]
controlnet_image


  0%|          | 0/20 [00:00<?, ?it/s]